# Setup

In [1]:
import os
from time import sleep
import openai
from openai import OpenAI

openai.api_key = os.environ["OPENAI_API_KEY"]

client = OpenAI()


def llm(prompt, stop=["\n"]):
    try:
        chat_completion = client.chat.completions.create(
            # model="gpt-3.5-turbo",
            # model="gpt-4-turbo-preview",
            # model="gpt-4-32k",
            model="gpt-4-1106-preview",
            # messages=[{"role": "user", "content": prompt}],
            # messages=[{"role": "user", "content": prompt}, {"role": "system", "content": "You are a serious system and deep thinker, do not return the answer ambiguous answered the question, do not mix the meaning of found and acquire a company."}],
            messages=[{"role": "user", "content": prompt}, {"role": "system", "content": "You are a serious deep thinker, help answer task accurately. Do not return the answer ambiguous answered the question. Make sure the answer accurately answer the initial question. When handling questions and answers, be flexible with synonyms for accuracy and depth.\
                                                            Yet, strictly differentiate terms with similar but distinct meanings without interchangeability. Do not try to correct the question and get a answer, just say unknown if there's no perfect answer for the question \
                                                            The check stage is necessary before returning the answer"}],
            temperature=0,
            max_tokens=100,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop
        )
    except:
        print("Error in completion, try sleep 5 minutes")
        sleep(300)
        chat_completion = client.chat.completions.create(
            # model="gpt-3.5-turbo",
            model="gpt-4-turbo-preview",
            # model="gpt-4-32k",
            # model="gpt-4-1106-preview",
            # messages=[{"role": "user", "content": prompt}],
            # messages=[{"role": "user", "content": prompt}, {"role": "system", "content": "You are a serious system and deep thinker, do not return the answer ambiguous answered the question, do not mix the meaning of found and acquire a company."}],
            messages=[{"role": "user", "content": prompt}, {"role": "system", "content": "You are a serious deep thinker, help answer task accurately. Do not return the answer ambiguous answered the question. Make sure the answer accurately answer the initial question. When handling questions and answers, be flexible with synonyms for accuracy and depth.\
                                                            Yet, strictly differentiate terms with similar but distinct meanings without interchangeability. Do not try to correct the question and get a answer, just say unknown if there's no perfect answer for the question \
                                                            The check stage is necessary before returning the answer"}],
            temperature=0,
            max_tokens=100,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop
        )

    last_message = chat_completion.choices[0].message.content
    return last_message

# deprecated code:
# def llm(prompt, stop=["\n"]):
#     response = openai.Completion.create(
#       model="text-davinci-002",
#       prompt=prompt,
#       temperature=0,
#       max_tokens=100,
#       top_p=1,
#       frequency_penalty=0.0,
#       presence_penalty=0.0,
#       stop=stop
#     )
#     return response["choices"][0]["text"]

In [2]:
import wikienv, wrappers
env = wikienv.WikiEnv()
env = wrappers.HotPotQAWrapper(env, split="dev")
env = wrappers.LoggingWrapper(env)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

In [3]:
#naive_poc

import json
import sys
from time import sleep

folder = './prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

webthink_examples = prompt_dict['webthink_simple6']
instruction = """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be four types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar or related entities.
(2) Lookup[keyword], which returns the next sentence containing keyword or semantic meaning words in the current passage.
(3) Check[answer] Before providing the answer, please carefully review and verify the accuracy of every term and expression in the original question. If the answer is ambiguous or does not fully address the question, please return more similar entities to search.
(4) Finish[answer], which returns the answer and finishes the task.
Here are some examples.
"""
webthink_prompt = instruction + webthink_examples

# print(webthink_prompt)

def webthinkRecurr(idx=None, prompt=webthink_prompt, to_print=True):
    question = env.reset(idx=idx)
    # if to_print:
    #     print("Poc question", question)
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        thought_action = llm(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        # print('thought_action', thought_action)
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            # print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
        # print("action output start")
        # print(action[0].lower())
        # print(action[1:])
        # print("action output end")
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

# # question = 'When was the  Argentine former footballer which  Dutch manager Ronald Koeman replaced in  2014-15 Southampton F.C. season born'
# question = 'What company did Rex Maughan aquire?'
# # Forever Living Products
# # Rex Maughan
# r, info = webthinkRecurr(question, to_print=True)

In [4]:
import random
import time
idxs = list(range(7405))
random.Random(233).shuffle(idxs)

rs = []
infos = []
old_time = time.time()
for i in idxs[:500]:
    r, info = webthinkRecurr(i, to_print=True)
    rs.append(info['em'])
    infos.append(info)
    print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
    print('-----------')
    print()

Thought 1: Thought 1: To answer this question, I need to identify the American action crime thriller film directed by Stuart Baird that Irene Jacob completed a movie before. However, the question does not specify which film by Stuart Baird it is referring to. I will need to search for Stuart Baird's filmography to find the relevant film and then determine which movie Irene Jacob completed before that film.

Action 1: Search[Stuart Baird filmography]
Observation 1: Could not find Stuart Baird filmography. Similar: ['Michael Tronick (section Filmography)', 'Rebecca Romijn (section Filmography)', 'Jon Hamm', 'James Cosmo (section Filmography)', 'Jeff Imada (section Filmography)'].

Thought 2: Thought 2: Since I could not find Stuart Baird's filmography directly, I need to search for Stuart Baird and then look for his filmography within the information available about him to identify the American action crime thriller film in question.

Action 2: Search[Stuart Baird]
Observation 2: Stuart 

KeyboardInterrupt: 